In [2]:
import pandas as pd

def read_instruments_to_dataframe(file_path):
    # Read the JSON file into a pandas DataFrame
    df = pd.read_json(file_path)
    
    # Optionally, perform any necessary data cleaning or transformations here
    # For example, you might want to convert string dates to datetime objects:
    # df['date_column'] = pd.to_datetime(df['date_column'])
    
    return df

instruments_file_path = 'instruments.json'  # Adjust the file path if necessary
instruments_df = read_instruments_to_dataframe(instruments_file_path)

# Now you can use instruments_df to perform various analyses or data manipulations
print(instruments_df.head())  # Display the first few rows of the DataFrame



   instrument_token  exchange_token   tradingsymbol    name  last_price  \
0         258770438         1010822  EURINR24APRFUT  EURINR           0   
1         262676230         1026079  EURINR24AUGFUT  EURINR           0   
2         268449030         1048629  EURINR24DECFUT  EURINR           0   
3         261799430         1022654  EURINR24JULFUT  EURINR           0   
4         260801030         1018754  EURINR24JUNFUT  EURINR           0   

       expiry  strike  tick_size  lot_size instrument_type  segment exchange  
0  2024-04-26     0.0     0.0025         1             FUT  BCD-FUT      BCD  
1  2024-08-28     0.0     0.0025         1             FUT  BCD-FUT      BCD  
2  2024-12-27     0.0     0.0025         1             FUT  BCD-FUT      BCD  
3  2024-07-29     0.0     0.0025         1             FUT  BCD-FUT      BCD  
4  2024-06-26     0.0     0.0025         1             FUT  BCD-FUT      BCD  


In [13]:
from datetime import datetime

# Ensure expiry is in datetime format
instruments_df['expiry'] = pd.to_datetime(instruments_df['expiry'])

# Get the current year and month
current_year = datetime.now().year
current_month = datetime.now().month

# Filter for options and expiry within the current month
options_df = instruments_df[
                            (instruments_df['expiry'].dt.year == current_year) &
                            (instruments_df['name'] == 'SBIN') &
                            (instruments_df['expiry'].dt.month == current_month)]

print(options_df.head())  # Display the first few rows of the filtered DataFrame


       instrument_token  exchange_token   tradingsymbol  name  last_price  \
35779          17026050           66508    SBIN24MARFUT  SBIN           0   
66552          49449474          193162  SBIN24MAR505PE  SBIN           0   
66553          49449730          193163  SBIN24MAR510CE  SBIN           0   
66554          49449986          193164  SBIN24MAR510PE  SBIN           0   
66555          49453058          193176  SBIN24MAR540PE  SBIN           0   

          expiry  strike  tick_size  lot_size instrument_type  segment  \
35779 2024-03-28     0.0       0.05      1500             FUT  NFO-FUT   
66552 2024-03-28   505.0       0.05      1500              PE  NFO-OPT   
66553 2024-03-28   510.0       0.05      1500              CE  NFO-OPT   
66554 2024-03-28   510.0       0.05      1500              PE  NFO-OPT   
66555 2024-03-28   540.0       0.05      1500              PE  NFO-OPT   

      exchange  
35779      NFO  
66552      NFO  
66553      NFO  
66554      NFO  
66555  

In [15]:
# Placeholder: Assume we have the current price of SBIN stock. You need to replace this with actual fetching logic.
current_price_sbin = 788  # Example current price

# Calculate the absolute difference between each option's strike price and the current stock price
options_df['strike_diff'] = options_df['strike'].apply(lambda x: abs(x - current_price_sbin))

# Find the minimum strike difference to get the closest strike price to the current stock price
min_strike_diff = options_df['strike_diff'].min()

# Filter for options with the closest strike price (ATM) and are of type CE or PE
atm_options_df = options_df[(options_df['strike_diff'] == min_strike_diff) & 
                            ((options_df['instrument_type'] == 'CE') | (options_df['instrument_type'] == 'PE'))]

print(atm_options_df[['tradingsymbol', 'name', 'expiry', 'strike', 'instrument_type']].head())  # Display relevant columns


        tradingsymbol  name     expiry  strike instrument_type
66637  SBIN24MAR790CE  SBIN 2024-03-28   790.0              CE
66638  SBIN24MAR790PE  SBIN 2024-03-28   790.0              PE


/var/folders/8h/q83gsn1124x2qr7qvkswq39m0000gn/T/ipykernel_27669/2833611430.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  options_df['strike_diff'] = options_df['strike'].apply(lambda x: abs(x - current_price_sbin))


In [22]:
# Filter for options and expiry within the current month, and make a copy to avoid SettingWithCopyWarning
options_df = instruments_df[
                            (instruments_df['expiry'].dt.year == current_year) &
                            (instruments_df['name'] == 'SBIN') &
                            (instruments_df['expiry'].dt.month == current_month)
                            ].copy()

# Assume we have the current price of SBIN stock
current_price_sbin = 788  # Example current price

# Calculate the absolute difference between each option's strike price and the current stock price
options_df['strike_diff'] = options_df['strike'].apply(lambda x: abs(x - current_price_sbin))

# Find the minimum strike difference to get the closest strike price to the current stock price
min_strike_diff = options_df['strike_diff'].min()

# Filter for options with the closest strike price (ATM) and are of type CE or PE
atm_options_df = options_df[(options_df['strike_diff'] == min_strike_diff) & 
                            ((options_df['instrument_type'] == 'CE') | (options_df['instrument_type'] == 'PE'))]

print(atm_options_df[['tradingsymbol', 'name', 'expiry', 'strike', 'instrument_type','tick_size']].head())  # Display relevant columns


        tradingsymbol  name     expiry  strike instrument_type  tick_size
66637  SBIN24MAR790CE  SBIN 2024-03-28   790.0              CE       0.05
66638  SBIN24MAR790PE  SBIN 2024-03-28   790.0              PE       0.05
